# Imports

In [1]:
import os
import sys

# Paths

In [2]:
NOTEBOOKS_PATH: str = os.getcwd()
SETTINGS_PATH: str = NOTEBOOKS_PATH.replace("notebooks", "settings")
RESULTS_PATH: str = os.path.join(NOTEBOOKS_PATH, "results")
QSRR_IC_PATH: str = os.path.dirname(NOTEBOOKS_PATH)

OPTIMIZATION_PATH: str = os.path.join(SETTINGS_PATH, "optimization.json")

In [3]:
for path_ in {NOTEBOOKS_PATH, QSRR_IC_PATH, SETTINGS_PATH}:
    print(f"Path : {path_}")

Path : C:\Users\petar\PycharmProjects\QSRR_IC
Path : C:\Users\petar\PycharmProjects\QSRR_IC\notebooks
Path : C:\Users\petar\PycharmProjects\QSRR_IC\settings


In [4]:
if QSRR_IC_PATH not in sys.path:
    sys.path.append(QSRR_IC_PATH)

In [5]:
# Change the working dir to QSRR IC path
os.chdir(QSRR_IC_PATH)

# QSRR IC module imports

In [6]:
from qsrr_ic.config import QsrrIcConfig
from qsrr_ic.models.qsrr.domain_models import QsrrData
from qsrr_ic.load import (
    load_dataset,
    QsrrIcData, 
    QsrrIcDataset
)
from qsrr_ic.process import ProcessData
from qsrr_ic.runners import QsrrOptimizerRunner
from qsrr_ic.optimization.domain_models import OptimizerSettings

# Optimization

This example shows how use the APIs to optimize hyper-parameters of quantitative structure-retention relationships (QSRR) models and analyze the results. Differential Evolution is used for all regressors with more than 1 hyper-parameter (all except PLS). PLS is optimized using a gradient-based approach. Knee point of the plot of RMSECV vs n(LVs) is found and represents the optimal n(LVs).

## Instantiate `QsrrIcConfig` object using the "optimization.json" example
- `QsrrIcConfig` is an object used to load all configs & settings
- Can be instantiated in three ways:
  * Using the constructor, takes individual config classes, as attributes
  * Using the factory method `QsrrIcConfig::from_dict()` which takes a dictionary of configs/settings
  * Using the method `QsrrIcConfig::from_json(filename="filename.json")` that loads a dictionary of config/settings & instantiates `QsrrIcConfig` using the factory method `QsrrIcConfig::from_dict()`

In [7]:
optimization_config: QsrrIcConfig = QsrrIcConfig.from_json(filename=OPTIMIZATION_PATH)

In [8]:
optimization_config.to_dict()

{'dataset': {'molecular_descriptors_for_qsrr_training_path': './datasets/qsrr_ic/2025-01-05-molecular_descriptors_for_qsrr_training.csv',
  'isocratic_retention_path': './datasets/qsrr_ic/2025-01-05-isocratic_retention.csv',
  'molecular_descriptors_for_iso2grad_path': './datasets/qsrr_ic/2025-01-05-molecular_descriptors_for_iso2grad.csv',
  'gradient_void_times_path': './datasets/qsrr_ic/2025-01-05-gradient_void_times.csv',
  'gradient_profiles_path': './datasets/qsrr_ic/2025-01-05-gradient_profiles.csv',
  'gradient_retention_path': './datasets/qsrr_ic/2025-01-05-gradient_retention.csv'},
 'training_type': 'optimization',
 'train_test_split': {'test_ratio': 0.7, 'random_seed': 0, 'shuffle': True},
 'hyper_parameters': {'PLS': {'n_components': [1, 10]},
  'xGB': {'n_estimators': [10, 500],
   'learning_rate': [0.1, 0.9],
   'max_depth': [1, 5]},
  'GBR': {'n_estimators': [10, 500],
   'learning_rate': [0.1, 0.9],
   'max_depth': [1, 5]},
  'RFR': {'n_estimators': [10, 500],
   'max_de

## Load and Prepare Data

In [9]:
# Load the dataset
dataset: QsrrIcDataset = load_dataset(optimization_config.dataset_config)

# Process dataset for training
data: QsrrIcData = ProcessData(dataset).process()

# Prepare & split QSRR data to train/test
qsrr_data: QsrrData = QsrrData(
    y=data.isocratic_retention,
    x=data.molecular_descriptors_for_qsrr_training
)
optimization_config.train_test_split_config.random_seed = 0
qsrr_train_data, qsrr_test_data = qsrr_data.split(optimization_config.train_test_split_config)

## Optimize QSRR models

Configuration class `QsrrIcConfig`, among others, contains an dictionary of `RegressorType`:`HyperParameterConfig` key:value pairs. `HyperParameterConfig` contains an instance of the HyperParameterRegistry that stores hyper-parameters (for single train) or hyper-parameter ranges (for optimization). 

Package *qsrr_ic* in the *QSRR_IC* repository implements singleton ModelRunner classes to run the model optimization.

Below, we loop over the `RegressorType` & the respective `HyperParameterConfigs`, then instantiate `QsrrOptimizerRunner` that takes an instance of  `OptimizerSetting` and `QsrrData` as inputs and returns an instance of ˙`QsrrModelOptimizer` and an instance of `OptimizerResults` with optimal hyper-parameters. It uses `QsrrModelOptimizer` for optimization. `QsrrModelOptimizer` can also be run without the runner class.

In [10]:
optimizers = {}
optimizer_results = {}


for regressor_type, hyper_parameter_config in optimization_config.hyper_parameter_config.items():

    print(f"Running optimization of the {regressor_type.name} QSRR model...")
    
    optimizer_settings = OptimizerSettings(
        regressor_type,
        hyper_parameter_config.hyper_parameter_registry,
        cv_settings=optimization_config.cross_validation_config.cv_settings,
        global_search_settings=optimization_config.global_search_config.global_search_settings
    )

    optimizer_runner = QsrrOptimizerRunner()

    optimizer, optimizer_results_ = optimizer_runner.run(
        optimizer_settings=optimizer_settings,
        qsrr_train_data=qsrr_train_data,
        qsrr_test_data=qsrr_test_data
    )

    optimizers[regressor_type] = optimizer
    optimizer_results[regressor_type] = optimizer_results_

Running optimization of the PLS QSRR model...
Running optimization of the xGB QSRR model...
differential_evolution step 1: f(x)= 0.749517
Polishing solution with 'L-BFGS-B'
Running optimization of the GBR QSRR model...


C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was pass

differential_evolution step 1: f(x)= 0.753145
Polishing solution with 'L-BFGS-B'


C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was pass

Running optimization of the RFR QSRR model...


...

C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vec

Running optimization of the ADA QSRR model...


C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A colu

differential_evolution step 1: f(x)= 0.753936
Polishing solution with 'L-BFGS-B'


C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\petar\PycharmProjects\QSRR_IC\.venv_3.8\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A colu